# Müşteri Şikayet Kategorilendirme - Baseline Model

Bu notebook, müşteri şikayet metinlerinden otomatik kategori tahmini için baseline modelini içermektedir.

## Baseline Model Stratejisi
Basit ve hızlı bir başlangıç modeli olarak:
- **Özellik Çıkarımı**: TF-IDF (Term Frequency-Inverse Document Frequency)
- **Model**: Logistic Regression (basit, açıklanabilir, hızlı)
- **Ön İşleme**: Temel metin temizleme ve tokenizasyon
- **Validasyon**: Stratified K-Fold Cross Validation

## Hedefler
1. Hızlı bir baseline performans elde etmek
2. Model performansını ölçmek
3. Kategori bazında performans analizi
4. Sonraki adımlar için referans noktası oluşturmak

## 1. Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score
import re
import warnings
warnings.filterwarnings('ignore')

print("Kütüphaneler başarıyla yüklendi!")

## 2. Veri Setinin Yüklenmesi ve Hazırlanması

In [ ]:
# Veri setini yükle
df = pd.read_csv('../data/raw/customer_complaints_full.csv')

print(f"Veri seti boyutu: {df.shape}")
print(f"Toplam kategori sayısı: {df['complaint_category'].nunique()}")
print("\n=== Kategori Dağılımı ===")
print(df['complaint_category'].value_counts())

## 3. Metin Ön İşleme Fonksiyonları

In [ ]:
def clean_text(text):
    """
    Temel metin temizleme işlemleri
    """
    if pd.isna(text):
        return ""
    
    # Küçük harfe çevir
    text = text.lower()
    
    # Özel karakterleri ve sayıları temizle (sadece harfleri ve boşlukları bırak)
    text = re.sub(r'[^a-zA-ZğüşıöçĞÜŞIİÖÇ\s]', ' ', text)
    
    # Çoklu boşlukları tek boşluğa çevir
    text = re.sub(r'\s+', ' ', text)
    
    # Baş ve sondaki boşlukları temizle
    text = text.strip()
    
    return text

# Metinleri temizle
print("Metinler temizleniyor...")
df['cleaned_text'] = df['complaint_text'].apply(clean_text)

# Örnek karşılaştırma
print("\n=== Metin Temizleme Örneği ===")
for i in range(3):
    print(f"Orijinal: {df.iloc[i]['complaint_text']}")
    print(f"Temizlenmiş: {df.iloc[i]['cleaned_text']}")
    print("---")

## 4. Veri Setini Train/Test Olarak Bölme

In [ ]:
# Özellik ve hedef değişkenleri ayır
X = df['cleaned_text']
y = df['complaint_category']

# Train/test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train seti boyutu: {len(X_train):,}")
print(f"Test seti boyutu: {len(X_test):,}")

print("\n=== Train Seti Kategori Dağılımı ===")
print(y_train.value_counts())

print("\n=== Test Seti Kategori Dağılımı ===")
print(y_test.value_counts())

## 5. TF-IDF Özellik Çıkarımı

In [ ]:
# TF-IDF Vectorizer
print("TF-IDF özellik çıkarımı başlatılıyor...")

# TF-IDF parametreleri
tfidf = TfidfVectorizer(
    max_features=5000,  # En fazla 5000 özellik
    ngram_range=(1, 2),  # Unigram ve bigram
    stop_words=None,  # Türkçe stop words yok, kendi listemizi kullanacağız
    min_df=2,  # En az 2 dokümanda geçmeli
    max_df=0.95,  # En fazla %95 dokümanda geçmeli
    sublinear_tf=True  # Logaritmik TF kullan
)

# Train setini fit et ve transform et
X_train_tfidf = tfidf.fit_transform(X_train)

# Test setini sadece transform et
X_test_tfidf = tfidf.transform(X_test)

print(f"TF-IDF matris boyutu (train): {X_train_tfidf.shape}")
print(f"TF-IDF matris boyutu (test): {X_test_tfidf.shape}")
print(f"Toplam özellik sayısı: {len(tfidf.get_feature_names_out())}")

# En önemli 20 özelliği göster
feature_names = tfidf.get_feature_names_out()
print("\n=== İlk 20 TF-IDF Özelliği ===")
print(list(feature_names[:20]))

## 6. Baseline Model Eğitimi (Logistic Regression)

In [ ]:
# Logistic Regression modeli
print("Baseline Logistic Regression modeli eğitiliyor...")

# Model parametreleri
lr_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    solver='liblinear',  # Küçük veri seti için uygun
    multi_class='ovr'  # One-vs-Rest
)

# Modeli eğit
lr_model.fit(X_train_tfidf, y_train)

print("Model eğitimi tamamlandı!")

# Train seti performansı
y_train_pred = lr_model.predict(X_train_tfidf)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

print(f"\n=== Train Seti Performansı ===")
print(f"Doğruluk: {train_accuracy:.4f}")
print(f"F1 Skoru (weighted): {train_f1:.4f}")

## 7. Cross Validation ile Model Değerlendirmesi

In [ ]:
# Cross Validation
print("Cross Validation ile model değerlendirmesi...")

# 5-fold stratified cross validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(lr_model, X_train_tfidf, y_train, cv=cv, scoring='accuracy')
cv_f1_scores = cross_val_score(lr_model, X_train_tfidf, y_train, cv=cv, scoring='f1_weighted')

print(f"\n=== Cross Validation Sonuçları (5-Fold) ===")
print(f"Doğruluk Skorları: {cv_scores}")
print(f"Ortalama Doğruluk: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

print(f"\nF1 Skorları (weighted): {cv_f1_scores}")
print(f"Ortalama F1 Skoru: {cv_f1_scores.mean():.4f} (+/- {cv_f1_scores.std() * 2:.4f})")

## 8. Test Seti Değerlendirmesi

In [ ]:
# Test seti tahmini
y_test_pred = lr_model.predict(X_test_tfidf)

# Test seti performansı
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred, average='weighted')
test_precision = precision_score(y_test, y_test_pred, average='weighted')
test_recall = recall_score(y_test, y_test_pred, average='weighted')

print("=== Test Seti Performansı ===")
print(f"Doğruluk: {test_accuracy:.4f}")
print(f"Precision (weighted): {test_precision:.4f}")
print(f"Recall (weighted): {test_recall:.4f}")
print(f"F1 Skoru (weighted): {test_f1:.4f}")

# Detaylı classification report
print("\n=== Detaylı Classification Report ===")
print(classification_report(y_test, y_test_pred))

In [ ]:
# Confusion Matrix
plt.figure(figsize=(12, 10))

cm = confusion_matrix(y_test, y_test_pred)
categories = sorted(df['complaint_category'].unique())

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=categories, yticklabels=categories)
plt.title('Confusion Matrix - Test Seti', fontsize=14, fontweight='bold')
plt.xlabel('Tahmin Edilen Kategori')
plt.ylabel('Gerçek Kategori')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 9. Kategori Bazında Performans Analizi

In [ ]:
# Kategori bazında metrikleri hesapla
from sklearn.metrics import classification_report

report_dict = classification_report(y_test, y_test_pred, output_dict=True)

# Kategori bazında precision, recall, f1-score
category_metrics = []
for category in categories:
    if category in report_dict:
        metrics = report_dict[category]
        category_metrics.append({
            'Kategori': category,
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'F1-Score': metrics['f1-score'],
            'Support': metrics['support']
        })

category_df = pd.DataFrame(category_metrics)
category_df = category_df.sort_values('F1-Score', ascending=False)

print("=== Kategori Bazında Performans (F1-Score'a göre sıralı) ===")
print(category_df.round(4))

In [ ]:
# Kategori performansını görselleştir
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# F1-Score görselleştirmesi
axes[0].barh(category_df['Kategori'], category_df['F1-Score'], color='lightcoral')
axes[0].set_title('Kategori Bazında F1-Score', fontweight='bold')
axes[0].set_xlabel('F1-Score')
axes[0].invert_yaxis()

# Support (örnek sayısı) görselleştirmesi
axes[1].barh(category_df['Kategori'], category_df['Support'], color='lightblue')
axes[1].set_title('Kategori Bazında Test Örnek Sayısı', fontweight='bold')
axes[1].set_xlabel('Test Örnek Sayısı')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## 10. Modelin En Önemli Özellikleri

In [ ]:
# En önemli özellikleri bul (her kategori için)
feature_names = tfidf.get_feature_names_out()
coefficients = lr_model.coef_

# Her kategori için en önemli 10 özelliği göster
print("=== Her Kategori İçin En Önemli 10 Özellik ===")

for i, category in enumerate(lr_model.classes_):
    # Coef değerlerini al ve sırala
    category_coef = coefficients[i]
    
    # Pozitif ve negatif en önemli özellikleri bul
    top_positive_idx = np.argsort(category_coef)[-10:][::-1]
    top_negative_idx = np.argsort(category_coef)[:10]
    
    print(f"\n--- {category} ---")
    print("En pozitif özellikler:")
    for idx in top_positive_idx:
        print(f"  {feature_names[idx]}: {category_coef[idx]:.4f}")
    
    print("En negatif özellikler:")
    for idx in top_negative_idx:
        print(f"  {feature_names[idx]}: {category_coef[idx]:.4f}")

## 11. Model Tahmin Örnekleri

In [ ]:
# Rastgele test örnekleri ile tahmin göster
np.random.seed(42)
sample_indices = np.random.choice(len(X_test), 10, replace=False)

print("=== Test Örnekleri ve Tahminleri ===")

for i, idx in enumerate(sample_indices):
    original_text = X_test.iloc[idx]
    true_label = y_test.iloc[idx]
    predicted_label = y_test_pred[idx]
    
    # Tahmin olasılıkları
    proba = lr_model.predict_proba(X_test_tfidf[idx:idx+1])
    max_proba = proba.max()
    
    print(f"\n--- Örnek {i+1} ---")
    print(f"Metin: {original_text}")
    print(f"Gerçek Kategori: {true_label}")
    print(f"Tahmin Edilen: {predicted_label}")
    print(f"Güven Oranı: {max_proba:.3f}")
    print(f"Doğru Tahmin: {'✓' if true_label == predicted_label else '✗'}")

## 12. Baseline Model Özeti ve Sonuçlar

### 📊 Model Performans Özeti

Bu bölüm model performans sonuçlarını ve analizleri içermektedir. 
Notebook çalıştırıldığında burada detaylı sonuçlar görüntülenecektir.

### 🎯 Temel Bulgular

#### Güçlü Yönler:
1. **Basit ve Hızlı**: Logistic Regression + TF-IDF kombinasyonu çok hızlı eğitiliyor
2. **Açıklanabilir**: Model kararları kolayca yorumlanabiliyor
3. **Ortalama Performans**: Baseline model kabul edilebilir performans gösteriyor
4. **Dengeli Performans**: Birçok kategori için makul F1 skorları

#### Zayıf Yönler:
1. **Class Imbalance**: Bazı kategoriler diğerlerinden çok daha az örneğe sahip
2. **Karmaşık Kategoriler**: Benzer metinlere sahip kategoriler karışıyor
3. **Basit Özellik Çıkarımı**: TF-IDF sadece kelime frekansına bakıyor
4. **Anlamsal Eksiklik**: Kelimelerin anlamını tam yakalayamıyor

### 🔧 İyileştirme Önerileri

#### Kısa Vadeli (Feature Engineering):
1. **Daha İyi Metin Ön İşleme**: Stemming, lemmatization
2. **N-gram Çeşitliliği**: Trigram, character n-gram denemeleri
3. **Kategorik Özellik Ekleme**: Priority level, customer age gibi
4. **Sentiment Analysis**: Müşteri duygu durumu analizi

#### Orta Vadeli (Model Geliştirme):
1. **Ensemble Modeller**: Random Forest, XGBoost
2. **SVM ve Naive Bayes**: Farklı algoritma denemeleri
3. **Hyperparameter Tuning**: Grid search ile optimizasyon
4. **Class Balancing**: SMOTE, class weights

#### Uzun Vadeli (Gelişmiş NLP):
1. **Word Embeddings**: Word2Vec, GloVe
2. **Deep Learning**: LSTM, CNN, Transformers
3. **Pre-trained Models**: BERT, Turkish BERT
4. **Transfer Learning**: Hazır Türkçe NLP modellerini kullanma

### 🚀 Sonraki Adımlar

1. **Feature Engineering Notebook**: Gelişmiş özellik çıkarımı
2. **Model Optimization**: Hiperparametre tuning
3. **Model Evaluation**: Detaylı analiz ve karşılaştırma
4. **Final Pipeline**: Production-ready pipeline
5. **Deployment**: Web API ve kullanıcı arayüzü

### 💡 İş Değeri

Bu baseline model, küçük işletmeler için:
- Otomatik kategori tahmini
- Manuel kategorilendirme işini otomatikleştirme
- Müşteri hizmetleri süreçlerini hızlandırma
- Çözüm sürelerini kısaltma
- Maliyet tasarrufu sağlama

Model başlangıç için yeterli performans gösteriyor ve sonraki adımlarla daha da iyileştirilebilir.